In [ ]:
# Convert xml files to yolo txt files
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join

In [ ]:
dir_path = 'dataset_labelledimage'
im_path = 'dataset_labelledimage'
classes = ['hard exudates', 'dot haemorrhage', 'blot haemorrhage' , 'preretinal' ,'fibrosis' , 'new vessels' , 'venous beading' ]
with open ('obj.names', 'w') as f:
    for clas in classes:
        f.write('%s\n' % clas)
        
f.close()

In [ ]:
def getImagesInDir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '/*.jpg'):
        image_list.append(filename)

    return image_list

In [ ]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = "{:.6f}".format(x*dw)
    w = "{:.6f}".format(w*dw)
    y = "{:.6f}".format(y*dh)
    h = "{:.6f}".format(h*dh)
    return (x,y,w,h)

In [ ]:
def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]
    try:
        in_file = open(dir_path + '/' + basename_no_ext + '.xml')
        out_file = open(output_path + '/' + basename_no_ext + '.txt', 'w')
        tree = ET.parse(in_file)
        root = tree.getroot()
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)

        for obj in root.iter('object'):
            difficult = obj.find('difficult').text
            cls = obj.find('name').text
            if cls not in classes or int(difficult)==1:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert((w,h), b)
            out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
        out_file.close()
    except:
        print(basename_no_ext + '.xml' + " File not Found.")
        out_file = open(output_path + '/' + basename_no_ext + '.txt', 'w').close()

In [ ]:
cwd = getcwd()
#xml_dir_path = cwd + '/' + dir_path
#full_dir_path = cwd + '/' + im_path
#output_path = cwd +'/yolo'


xml_dir_path =  dir_path
full_dir_path = im_path
output_path = im_path

if not os.path.exists(output_path):
    os.makedirs(output_path)

image_paths = getImagesInDir(full_dir_path)
list_file = open(cwd + '/image_list.txt', 'w')

for image_path in image_paths:
    list_file.write(image_path + '\n')
    convert_annotation(xml_dir_path, output_path, image_path)
list_file.close()

print("Finished processing: " + dir_path)

In [ ]:
with open('obj.data', 'w') as f:
    f.write('classes = %s\n' % len(classes))
    f.write('train = %s\nvalid = %s\nnames = %s\nbackup = %s' % (str(cwd + '/image_list.txt'), str(cwd + '/image_list.txt') , str(cwd + '/' + 'obj.names') , str(cwd + '/' + 'weight') ))
    
f.close()

backup = cwd +'/weight'

if not os.path.exists(backup):
    os.makedirs(backup)
    

print ("!./darknet detector train %s %s -clear -map" % (str(cwd + '/' + 'obj.data'), str(cwd + '/' + 'model.cfg')))    